In [ ]:
import os
os.chdir("..")

In [ ]:
from core.nn.LSTMmodel import LSTM
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import pickle
import datetime
import seaborn as sns
import covsirphy as cs
sns.set()
%matplotlib inline

%load_ext autoreload
%autoreload 2

In [ ]:
COUNTRY      = 'Greece'
DEVICE       = 'cpu'
TRAIN_UP_TO  = pd.to_datetime('2020-10-01')
ThreshConf   = 68
ThreshDead   = 20
target       = "New Confirmed"
FUTURE_DAYS  = 45

In [ ]:
lstm = LSTM(COUNTRY, TRAIN_UP_TO, FUTURE_DAYS, ThreshDead, target)

In [ ]:
df = lstm.init_data()

In [ ]:
lstm.iterations = 2
best = lstm.optimizeTreshold()
lstm.iterations = 15

In [ ]:
ThreshConf   = best["Threshold"]
lstm.simulate(ThreshConf = ThreshConf)

In [ ]:
from core.SIR import SIR

In [ ]:
sir_sim = SIR(COUNTRY, TRAIN_UP_TO)

In [ ]:
snl = sir_sim.init_data()

In [ ]:
summary = snl.summary()
summary["Start_dt"] = pd.to_datetime(summary["Start"], format="%d%b%Y")
summary["End_dt"] = pd.to_datetime(summary["End"], format="%d%b%Y")
query = summary[summary["End_dt"] > TRAIN_UP_TO]
all_phases = query.index.tolist()

snl.combine(phases=all_phases)
target_date = datetime.datetime.strftime(TRAIN_UP_TO - datetime.timedelta(days=1), format="%d%b%Y")
snl.separate(target_date)

In [ ]:
summary = snl.summary()

In [ ]:
summary = snl.summary()
all_phases = summary.index.tolist()
snl.disable(phases=all_phases[:-1])
snl.enable(phases=all_phases[-1:])
snl.summary()

In [ ]:
snl.estimate(model=cs.SIRF)

In [ ]:
df = snl.simulate();
df["New Confirmed"] = df["Confirmed"].diff()

In [ ]:
lstm.plot(SIRdata=df)

In [ ]:
accuracy, dfOverTime = lstm.accuracy(SIRdata=df)
cm = sns.color_palette("coolwarm", as_cmap=True)
accuracy.style.background_gradient(cmap = cm, vmin=np.min(accuracy.values), vmax=np.max(accuracy.values))

In [ ]:
dfOverTime.plot(title="Error over time. Lower values indicate higher accuracy.", xlabel="Days", ylabel="RMSE");